In [1]:
# copied from Agnostic_LR_Adult.py

import time
import numpy as np
import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.autograd import Variable
from numpy import genfromtxt
from Adult_dataset import MyDataset
from alpha_maximization import compute_alpha_with_fairness, compute_alpha_no_fairness, compute_alpha_with_local_fairness
import kernel
from risk_difference_calculation import risk_difference_calculation

In [4]:
batch_size = 30000
learning_rate = 0.1
num_epochs = 10
filename = 'adult_norm.csv'
numpy_data = genfromtxt(filename, delimiter=',')


train = []

index = batch_size
train_data = numpy_data[:, 1:-1][0:index]
train_label = numpy_data[:, -1][0:index]

test_data = numpy_data[:, 1:-1][index:]
test_label = numpy_data[:, -1][index:]

train_dataset = MyDataset(train_data, train_label)
test_dataset = MyDataset(test_data, test_label)
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = False)
test_loader = DataLoader(test_dataset, batch_size = len(numpy_data) - batch_size, shuffle = False)

class logsticRegression(nn.Module):
    def __init__(self, in_dim, n_class):
        super(logsticRegression, self).__init__()
        self.logstic = nn.Linear(in_dim, n_class)

    def forward(self, x):
        out = self.logstic(x)
        Theta_X = out
        out = torch.sigmoid(out)
        # return class labels, probas
        return out, Theta_X


def loss_no_fair(output, target):
    # cross entropy -\sum p(x) log q(x) 
    loss = torch.mean((-target * torch.log(output)- (1 - target) * torch.log(1 - output))) 

    return loss

# Zafar et al. Need to add some details to make it work better: cross validation to choose the hyperparameter
# theta_x is the distance to the boundary
def loss_with_fair(output, target, Theta_X, Sen, Sen_bar):
    # cross entropy
    pred_loss = torch.mean((-target * torch.log(output)- (1 - target) * torch.log(1 - output)))
    # Zafar et al. 
    fair_loss = torch.mul(Sen - Sen_bar, Theta_X)
    fair_loss = torch.mean(torch.mul(fair_loss, fair_loss))
    # they fixed the lagangian multiplier manually instead of using cross validation to compute lambda
    # may get better performance with cross validation
    return pred_loss + 4.0*fair_loss

# k is the number of clients
def loss_with_local_fair(output, target, Theta_X, Sen, k):
    pred_loss = torch.mean((-target * torch.log(output)- (1 - target) * torch.log(1 - output)))
    local_length = len(Sen) // k
    fair_loss = 0
    for i in range(k):
        Sen_bar_temp = torch.sum(Sen[i*local_length: (i + 1)*local_length])/local_length
        Sen_temp = Sen[i*local_length: (i + 1)*local_length]
        Theta_X_temp = Theta_X[i*local_length: (i + 1)*local_length]
        loss_temp = torch.mul(Sen_temp - Sen_bar_temp, Theta_X_temp)
        loss_temp = torch.mean(torch.mul(loss_temp, loss_temp))
        fair_loss = fair_loss + loss_temp
    print(pred_loss)
    print(fair_loss)
    return pred_loss + 1.0*fair_loss



def agnostic_loss_no_fair(output, target, weight):

    pred_loss = (-target * torch.log(output)- (1 - target) * torch.log(1 - output))
    pred_loss = torch.mean(torch.mul(pred_loss, weight))
    return pred_loss


def loss_with_agnostic_local_fair(output, target, Theta_X, weight, Sen, Sen_bar, k):
    pred_loss = (-target * torch.log(output)- (1 - target) * torch.log(1 - output))
    pred_loss = torch.mean(torch.mul(pred_loss, weight))

    fair_loss = 0
    local_length = len(Sen) // k
    for i in range(k):
        weight_temp = weight[i*local_length: (i+1)*local_length]
        Sen_temp = Sen[i*local_length: (i+1)*local_length]
        Theta_X_temp = Theta_X[i*local_length: (i+1)*local_length]
        temp_loss = torch.mul(torch.mul(weight_temp, Sen_temp) - Sen_bar[i], Theta_X_temp)
        fair_loss = fair_loss + torch.mean(torch.mul(temp_loss, temp_loss))

    return pred_loss + 2.0*fair_loss

# Agnostic 
def loss_with_agnostic_fair(output, target, Theta_X, weight, Sen, Sen_bar):
    pred_loss = (-target * torch.log(output)- (1 - target) * torch.log(1 - output))
    pred_loss = torch.mean(torch.mul(pred_loss, weight))

    fair_loss = torch.mul(torch.mul(weight, Sen) - Sen_bar, Theta_X)
    fair_loss = torch.mean(torch.mul(fair_loss, fair_loss))

    return pred_loss + 2.0*fair_loss

input_dim = 40
output_dim = 1

model = logsticRegression(input_dim, output_dim)
use_gpu = torch.cuda.is_available()


criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)


def compute_model_parameter_fairness(weight, Sen, ratio, option, k):
    for epoch in range(num_epochs):
        print('*' * 10)
        print(f'epoch {epoch+1}')
        running_acc = 0.0
        
        # Sets the module in training mode
        model.train() 
        for i,  (data, target) in enumerate(train_loader):
            img, label = data, target

            label = label.view(len(label), -1)

            out, Theta_X = model(img) # Theta_X >0 then 1 else 0, Theta_X is the distance to the boundary

            if option == "no_fair":
                loss = loss_no_fair(out, label)
            elif option == "with_fair":
                loss = loss_with_fair(out, label, Theta_X, Sen, ratio)
            elif option == "agnostic_fair":
                loss = loss_with_agnostic_fair(out, label, Theta_X, weight, Sen, ratio)
            elif option == "agnostic_loss_no_fair":
                loss = agnostic_loss_no_fair(out, label, weight)
            elif option == "loss_with_local_fair":
                loss = loss_with_local_fair(out, label, Theta_X, Sen, k)
            elif option == "loss_with_agnostic_local_fair":
                loss = loss_with_agnostic_local_fair(out, label, Theta_X, weight, Sen, ratio, k)

            print(loss)

            pred = torch.where(out > 0.5, torch.tensor(1.0), torch.tensor(0.0))

            running_acc += torch.sum((pred==label))

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        Sen_train = numpy_data[:, 0][0:index].reshape(-1, 1)
        risk_difference = np.random.rand(k)
        for j in range(k):
            local_length = len(pred) // k
            Sen_train_j = Sen_train[j*local_length : (j + 1)*local_length]
            pred_j = pred[j*local_length : (j + 1)*local_length]
            label_j = label[j*local_length : (j + 1)*local_length]
            risk_difference[j] = risk_difference_calculation(Sen_train_j, pred_j)
            accuracy = torch.sum((pred_j==label_j))
            accuracy = accuracy.numpy()
            print(f'Party {i + 1} accuracy {accuracy / local_length:.6f}')
        running_acc = running_acc.numpy()
        global_risk_difference = risk_difference_calculation(Sen_train, pred)
        for i in range(k):
            print(f'Party {i + 1} risk difference {risk_difference[i]:.6f}')
        print(f'global training risk dfference: {global_risk_difference:.6f}')
        print(f'Finish {epoch + 1} training epoch, Acc: {running_acc / len(train_label):.6f}')
        
        # Sets the module in evaluation mode.
        model.eval()
        theta = model.logstic.weight
        bias = model.logstic.bias

        running_acc = 0.0
        for data in test_loader:
            img, label = data
            img = img.view(img.size(0), -1)
            with torch.no_grad():
                out, Theta_X = model(img)

            pred = torch.where(out > 0.5, torch.tensor(1.0), torch.tensor(0.0))
            label = label.view(len(label), -1)
            running_acc += torch.sum((pred == label))

        Sen_test = numpy_data[:, 0][index:].reshape(-1, 1)
        for i in range(k):
            length = len(Sen_test) // k
            Sen_train_j = Sen_test[i*length: (i+1)*length]
            pred_j = pred[i * length: (i + 1) * length]
            label_j = label[i * length: (i + 1) * length]
            risk_difference = risk_difference_calculation(Sen_train_j, pred_j)
            accuracy = torch.sum((pred_j == label_j)).numpy()
            print(f'Finish party {i + 1} test risk difference {risk_difference:.6f}')
            print(f'Party {i + 1} accuracy {accuracy / len(pred_j):.6f}')
        running_acc = running_acc.numpy()
        print(f'Finish {epoch + 1} testing epoch, Acc: {running_acc / len(test_label):.6f}')
        risk_difference = risk_difference_calculation(Sen_test[0:20000], pred[0:20000])
        print(f'Finish {epoch + 1} risk difference {risk_difference:.6f}')


    return theta.detach().numpy().transpose(), bias.detach().numpy()

Iteration = 20
sigma = 2
B = 5
Xtr = train_data
Ytr = train_label
ref_idx = np.random.randint(len(test_data), size = 30)
Xref = test_data[ref_idx, :]
Sen = numpy_data[:, 0][0:index].reshape(-1, 1)
Sen_torch = torch.from_numpy(Sen).float()

weight = torch.ones(batch_size, 1)
ratio = 0.1
options = ["no_fair", "with_fair", "agnostic_fair", "agnostic_loss_no_fair", "loss_with_local_fair", "loss_with_agnostic_local_fair"]
ratio_fair = np.sum(Sen) / len(Sen)
print(ratio_fair)

tau = 1000 # useless
k = 2
ratio = np.random.rand(k)
for i in range(k):
    local_length = len(Sen) // k
    ratio[i] = np.sum(Sen[i*local_length: (i+1)*local_length]) / local_length

0.6760333333333334
**********
epoch 1
tensor(0.6977, grad_fn=<MeanBackward0>)
Party 1 accuracy 0.460733
Party 1 accuracy 0.459733
Party 1 risk difference 0.191799
Party 2 risk difference 0.204090
global training risk dfference: 0.197946
Finish 1 training epoch, Acc: 0.460233
Finish party 1 test risk difference 0.147014
Party 1 accuracy 0.620155
Finish party 2 test risk difference 0.161210
Party 2 accuracy 0.620024
Finish 1 testing epoch, Acc: 0.620089
Finish 1 risk difference 0.154084
**********
epoch 2
tensor(0.6709, grad_fn=<MeanBackward0>)
Party 1 accuracy 0.618467
Party 1 accuracy 0.615067
Party 1 risk difference 0.162501
Party 2 risk difference 0.162547
global training risk dfference: 0.162529
Finish 2 training epoch, Acc: 0.616767
Finish party 1 test risk difference 0.028919
Party 1 accuracy 0.733675
Finish party 2 test risk difference 0.022508
Party 2 accuracy 0.730784
Finish 2 testing epoch, Acc: 0.732230
Finish 2 risk difference 0.025721
**********
epoch 3
tensor(0.6492, grad_

tensor(0.5440, grad_fn=<MeanBackward0>)
Party 1 accuracy 0.752933
Party 1 accuracy 0.748867
Party 1 risk difference 0.000000
Party 2 risk difference 0.000000
global training risk dfference: 0.000000
Finish 9 training epoch, Acc: 0.750900
Finish party 1 test risk difference 0.000000
Party 1 accuracy 0.757194
Finish party 2 test risk difference 0.000000
Party 2 accuracy 0.752069
Finish 9 testing epoch, Acc: 0.754631
Finish 9 risk difference 0.000000
**********
epoch 10
tensor(0.5418, grad_fn=<MeanBackward0>)
Party 1 accuracy 0.752933
Party 1 accuracy 0.748867
Party 1 risk difference 0.000000
Party 2 risk difference 0.000000
global training risk dfference: 0.000000
Finish 10 training epoch, Acc: 0.750900
Finish party 1 test risk difference 0.000000
Party 1 accuracy 0.757194
Finish party 2 test risk difference 0.000000
Party 2 accuracy 0.752069
Finish 10 testing epoch, Acc: 0.754631
Finish 10 risk difference 0.000000
**********
epoch 1
tensor(0.5398, grad_fn=<MeanBackward0>)
Party 1 accura

tensor(0.5153, grad_fn=<MeanBackward0>)
Party 1 accuracy 0.752933
Party 1 accuracy 0.748867
Party 1 risk difference 0.000000
Party 2 risk difference 0.000000
global training risk dfference: 0.000000
Finish 7 training epoch, Acc: 0.750900
Finish party 1 test risk difference 0.000000
Party 1 accuracy 0.757194
Finish party 2 test risk difference 0.000000
Party 2 accuracy 0.752069
Finish 7 testing epoch, Acc: 0.754631
Finish 7 risk difference 0.000000
**********
epoch 8
tensor(0.5140, grad_fn=<MeanBackward0>)
Party 1 accuracy 0.752933
Party 1 accuracy 0.748933
Party 1 risk difference 0.000000
Party 2 risk difference 0.000206
global training risk dfference: 0.000103
Finish 8 training epoch, Acc: 0.750933
Finish party 1 test risk difference 0.000000
Party 1 accuracy 0.757194
Finish party 2 test risk difference 0.000000
Party 2 accuracy 0.752069
Finish 8 testing epoch, Acc: 0.754631
Finish 8 risk difference 0.000000
**********
epoch 9
tensor(0.5128, grad_fn=<MeanBackward0>)
Party 1 accuracy 0

tensor(0.4958, grad_fn=<MeanBackward0>)
Party 1 accuracy 0.753133
Party 1 accuracy 0.749200
Party 1 risk difference 0.000328
Party 2 risk difference 0.000278
global training risk dfference: 0.000026
Finish 5 training epoch, Acc: 0.751167
Finish party 1 test risk difference 0.000783
Party 1 accuracy 0.757719
Finish party 2 test risk difference 0.000614
Party 2 accuracy 0.752201
Finish 5 testing epoch, Acc: 0.754960
Finish 5 risk difference 0.000086
**********
epoch 6
tensor(0.4949, grad_fn=<MeanBackward0>)
Party 1 accuracy 0.753133
Party 1 accuracy 0.749267
Party 1 risk difference 0.000328
Party 2 risk difference 0.000376
global training risk dfference: 0.000024
Finish 6 training epoch, Acc: 0.751200
Finish party 1 test risk difference 0.000783
Party 1 accuracy 0.757719
Finish party 2 test risk difference 0.000419
Party 2 accuracy 0.752332
Finish 6 testing epoch, Acc: 0.755026
Finish 6 risk difference 0.000184
**********
epoch 7
tensor(0.4940, grad_fn=<MeanBackward0>)
Party 1 accuracy 0

tensor(0.4811, grad_fn=<MeanBackward0>)
Party 1 accuracy 0.753533
Party 1 accuracy 0.750000
Party 1 risk difference 0.000018
Party 2 risk difference 0.000421
global training risk dfference: 0.000219
Finish 3 training epoch, Acc: 0.751767
Finish party 1 test risk difference 0.001150
Party 1 accuracy 0.758639
Finish party 2 test risk difference 0.000689
Party 2 accuracy 0.753383
Finish 3 testing epoch, Acc: 0.756011
Finish 3 risk difference 0.000924
**********
epoch 4
tensor(0.4804, grad_fn=<MeanBackward0>)
Party 1 accuracy 0.753533
Party 1 accuracy 0.750067
Party 1 risk difference 0.000018
Party 2 risk difference 0.000519
global training risk dfference: 0.000268
Finish 4 training epoch, Acc: 0.751800
Finish party 1 test risk difference 0.001345
Party 1 accuracy 0.758770
Finish party 2 test risk difference 0.000884
Party 2 accuracy 0.753515
Finish 4 testing epoch, Acc: 0.756142
Finish 4 risk difference 0.001119
**********
epoch 5
tensor(0.4797, grad_fn=<MeanBackward0>)
Party 1 accuracy 0

...

tensor(0.4536, grad_fn=<MeanBackward0>)
Party 1 accuracy 0.766200
Party 1 accuracy 0.764333
Party 1 risk difference 0.003997
Party 2 risk difference 0.007572
global training risk dfference: 0.005785
Finish 7 training epoch, Acc: 0.765267
Finish party 1 test risk difference 0.012695
Party 1 accuracy 0.769019
Finish party 2 test risk difference 0.010504
Party 2 accuracy 0.766916
Finish 7 testing epoch, Acc: 0.767967
Finish 7 risk difference 0.011623
**********
epoch 8
tensor(0.4532, grad_fn=<MeanBackward0>)
Party 1 accuracy 0.766600
Party 1 accuracy 0.764600
Party 1 risk difference 0.005971
Party 2 risk difference 0.008826
global training risk dfference: 0.007399
Finish 8 training epoch, Acc: 0.765600
Finish party 1 test risk difference 0.014253
Party 1 accuracy 0.768887
Finish party 2 test risk difference 0.012615
Party 2 accuracy 0.768099
Finish 8 testing epoch, Acc: 0.768493
Finish 8 risk difference 0.013460
**********
epoch 9
tensor(0.4528, grad_fn=<MeanBackward0>)
Party 1 accuracy 0

tensor(0.4476, grad_fn=<MeanBackward0>)
Party 1 accuracy 0.770867
Party 1 accuracy 0.768867
Party 1 risk difference 0.019156
Party 2 risk difference 0.023283
global training risk dfference: 0.021220
Finish 5 training epoch, Acc: 0.769867
Finish party 1 test risk difference 0.025915
Party 1 accuracy 0.773354
Finish party 2 test risk difference 0.029866
Party 2 accuracy 0.774143
Finish 5 testing epoch, Acc: 0.773749
Finish 5 risk difference 0.027916
**********
epoch 6
tensor(0.4473, grad_fn=<MeanBackward0>)
Party 1 accuracy 0.771267
Party 1 accuracy 0.769533
Party 1 risk difference 0.020143
Party 2 risk difference 0.025057
global training risk dfference: 0.022601
Finish 6 training epoch, Acc: 0.770400
Finish party 1 test risk difference 0.027089
Party 1 accuracy 0.773617
Finish party 2 test risk difference 0.031229
Party 2 accuracy 0.774800
Finish 6 testing epoch, Acc: 0.774208
Finish 6 risk difference 0.029184
**********
epoch 7
tensor(0.4470, grad_fn=<MeanBackward0>)
Party 1 accuracy 0

tensor(0.4426, grad_fn=<MeanBackward0>)
Party 1 accuracy 0.777467
Party 1 accuracy 0.777133
Party 1 risk difference 0.044076
Party 2 risk difference 0.048814
global training risk dfference: 0.046446
Finish 3 training epoch, Acc: 0.777300
Finish party 1 test risk difference 0.050552
Party 1 accuracy 0.777953
Finish party 2 test risk difference 0.054544
Party 2 accuracy 0.778873
Finish 3 testing epoch, Acc: 0.778413
Finish 3 risk difference 0.052574
**********
epoch 4
tensor(0.4424, grad_fn=<MeanBackward0>)
Party 1 accuracy 0.777733
Party 1 accuracy 0.777267
Party 1 risk difference 0.044865
Party 2 risk difference 0.049190
global training risk dfference: 0.047029
Finish 4 training epoch, Acc: 0.777500
Finish party 1 test risk difference 0.051327
Party 1 accuracy 0.777822
Finish party 2 test risk difference 0.055907
Party 2 accuracy 0.779530
Finish 4 testing epoch, Acc: 0.778676
Finish 4 risk difference 0.053641
**********
epoch 5
tensor(0.4421, grad_fn=<MeanBackward0>)
Party 1 accuracy 0

tensor(0.4384, grad_fn=<MeanBackward0>)
Party 1 accuracy 0.782600
Party 1 accuracy 0.780067
Party 1 risk difference 0.050209
Party 2 risk difference 0.057716
global training risk dfference: 0.053963
Finish 1 training epoch, Acc: 0.781333
Finish party 1 test risk difference 0.051660
Party 1 accuracy 0.780712
Finish party 2 test risk difference 0.063756
Party 2 accuracy 0.782683
Finish 1 testing epoch, Acc: 0.781698
Finish 1 risk difference 0.057728
**********
epoch 2
tensor(0.4382, grad_fn=<MeanBackward0>)
Party 1 accuracy 0.782267
Party 1 accuracy 0.779933
Party 1 risk difference 0.050900
Party 2 risk difference 0.057089
global training risk dfference: 0.053996
Finish 2 training epoch, Acc: 0.781100
Finish party 1 test risk difference 0.051651
Party 1 accuracy 0.780844
Finish party 2 test risk difference 0.062932
Party 2 accuracy 0.782814
Finish 2 testing epoch, Acc: 0.781829
Finish 2 risk difference 0.057314
**********
epoch 3
tensor(0.4380, grad_fn=<MeanBackward0>)
Party 1 accuracy 0

tensor(0.4348, grad_fn=<MeanBackward0>)
Party 1 accuracy 0.781467
Party 1 accuracy 0.780333
Party 1 risk difference 0.050502
Party 2 risk difference 0.053055
global training risk dfference: 0.051782
Finish 9 training epoch, Acc: 0.780900
Finish party 1 test risk difference 0.048135
Party 1 accuracy 0.782157
Finish party 2 test risk difference 0.058892
Party 2 accuracy 0.783865
Finish 9 testing epoch, Acc: 0.783011
Finish 9 risk difference 0.053535
**********
epoch 10
tensor(0.4346, grad_fn=<MeanBackward0>)
Party 1 accuracy 0.781600
Party 1 accuracy 0.780533
Party 1 risk difference 0.050699
Party 2 risk difference 0.052741
global training risk dfference: 0.051724
Finish 10 training epoch, Acc: 0.781067
Finish party 1 test risk difference 0.048135
Party 1 accuracy 0.782157
Finish party 2 test risk difference 0.058892
Party 2 accuracy 0.783865
Finish 10 testing epoch, Acc: 0.783011
Finish 10 risk difference 0.053535


In [6]:
# "no_fair"
for i in range(Iteration):
    # compute with the option "with fair" => Zafar et al.
    theta, bias = compute_model_parameter_fairness(weight, Sen_torch, ratio_fair, options[0], k)

**********
epoch 1
tensor(0.6011, grad_fn=<MeanBackward0>)
Party 1 accuracy 0.824067
Party 1 accuracy 0.825067
Party 1 risk difference 0.181428
Party 2 risk difference 0.178632
global training risk dfference: 0.180037
Finish 1 training epoch, Acc: 0.824567
Finish party 1 test risk difference 0.130498
Party 1 accuracy 0.820260
Finish party 2 test risk difference 0.128313
Party 2 accuracy 0.821180
Finish 1 testing epoch, Acc: 0.820720
Finish 1 risk difference 0.129410
**********
epoch 2
tensor(0.5843, grad_fn=<MeanBackward0>)
Party 1 accuracy 0.823200
Party 1 accuracy 0.817533
Party 1 risk difference 0.131403
Party 2 risk difference 0.133356
global training risk dfference: 0.132383
Finish 2 training epoch, Acc: 0.820367
Finish party 1 test risk difference 0.073331
Party 1 accuracy 0.799107
Finish party 2 test risk difference 0.080462
Party 2 accuracy 0.796085
Finish 2 testing epoch, Acc: 0.797596
Finish 2 risk difference 0.076894
**********
epoch 3
tensor(0.5706, grad_fn=<MeanBackward0>)

tensor(0.5018, grad_fn=<MeanBackward0>)
Party 1 accuracy 0.754000
Party 1 accuracy 0.750400
Party 1 risk difference 0.001653
Party 2 risk difference 0.002464
global training risk dfference: 0.002058
Finish 9 training epoch, Acc: 0.752200
Finish party 1 test risk difference 0.003132
Party 1 accuracy 0.759033
Finish party 2 test risk difference 0.001932
Party 2 accuracy 0.753777
Finish 9 testing epoch, Acc: 0.756405
Finish 9 risk difference 0.002532
**********
epoch 10
tensor(0.5003, grad_fn=<MeanBackward0>)
Party 1 accuracy 0.754000
Party 1 accuracy 0.750400
Party 1 risk difference 0.001653
Party 2 risk difference 0.002464
global training risk dfference: 0.002058
Finish 10 training epoch, Acc: 0.752200
Finish party 1 test risk difference 0.003132
Party 1 accuracy 0.759033
Finish party 2 test risk difference 0.001932
Party 2 accuracy 0.753777
Finish 10 testing epoch, Acc: 0.756405
Finish 10 risk difference 0.002532
**********
epoch 1
tensor(0.4989, grad_fn=<MeanBackward0>)
Party 1 accura

tensor(0.4820, grad_fn=<MeanBackward0>)
Party 1 accuracy 0.754333
Party 1 accuracy 0.751067
Party 1 risk difference 0.002237
Party 2 risk difference 0.002732
global training risk dfference: 0.002485
Finish 7 training epoch, Acc: 0.752700
Finish party 1 test risk difference 0.003711
Party 1 accuracy 0.759559
Finish party 2 test risk difference 0.003474
Party 2 accuracy 0.754434
Finish 7 testing epoch, Acc: 0.756996
Finish 7 risk difference 0.003594
**********
epoch 8
tensor(0.4811, grad_fn=<MeanBackward0>)
Party 1 accuracy 0.754400
Party 1 accuracy 0.751133
Party 1 risk difference 0.002534
Party 2 risk difference 0.003028
global training risk dfference: 0.002781
Finish 8 training epoch, Acc: 0.752767
Finish party 1 test risk difference 0.003711
Party 1 accuracy 0.759559
Finish party 2 test risk difference 0.003474
Party 2 accuracy 0.754434
Finish 8 testing epoch, Acc: 0.756996
Finish 8 risk difference 0.003594
**********
epoch 9
tensor(0.4803, grad_fn=<MeanBackward0>)
Party 1 accuracy 0

tensor(0.4684, grad_fn=<MeanBackward0>)
Party 1 accuracy 0.758467
Party 1 accuracy 0.756533
Party 1 risk difference 0.010249
Party 2 risk difference 0.013733
global training risk dfference: 0.011990
Finish 5 training epoch, Acc: 0.757500
Finish party 1 test risk difference 0.012853
Party 1 accuracy 0.763763
Finish party 2 test risk difference 0.013942
Party 2 accuracy 0.759296
Finish 5 testing epoch, Acc: 0.761529
Finish 5 risk difference 0.013394
**********
epoch 6
tensor(0.4678, grad_fn=<MeanBackward0>)
Party 1 accuracy 0.758800
Party 1 accuracy 0.756933
Party 1 risk difference 0.010940
Party 2 risk difference 0.014324
global training risk dfference: 0.012631
Finish 6 training epoch, Acc: 0.757867
Finish party 1 test risk difference 0.013244
Party 1 accuracy 0.763763
Finish party 2 test risk difference 0.014526
Party 2 accuracy 0.759427
Finish 6 testing epoch, Acc: 0.761595
Finish 6 risk difference 0.013882
**********
epoch 7
tensor(0.4672, grad_fn=<MeanBackward0>)
Party 1 accuracy 0

tensor(0.4580, grad_fn=<MeanBackward0>)
Party 1 accuracy 0.761400
Party 1 accuracy 0.760333
Party 1 risk difference 0.022544
Party 2 risk difference 0.023900
global training risk dfference: 0.023222
Finish 3 training epoch, Acc: 0.760867
Finish party 1 test risk difference 0.022998
Party 1 accuracy 0.766654
Finish party 2 test risk difference 0.025158
Party 2 accuracy 0.762843
Finish 3 testing epoch, Acc: 0.764748
Finish 3 risk difference 0.024078
**********
epoch 4
tensor(0.4575, grad_fn=<MeanBackward0>)
Party 1 accuracy 0.761533
Party 1 accuracy 0.760400
Party 1 risk difference 0.022438
Party 2 risk difference 0.023891
global training risk dfference: 0.023164
Finish 4 training epoch, Acc: 0.760967
Finish party 1 test risk difference 0.022794
Party 1 accuracy 0.766916
Finish party 2 test risk difference 0.025547
Party 2 accuracy 0.762580
Finish 4 testing epoch, Acc: 0.764748
Finish 4 risk difference 0.024170
**********
epoch 5
tensor(0.4570, grad_fn=<MeanBackward0>)
Party 1 accuracy 0

...

tensor(0.4378, grad_fn=<MeanBackward0>)
Party 1 accuracy 0.783267
Party 1 accuracy 0.780667
Party 1 risk difference 0.054733
Party 2 risk difference 0.062566
global training risk dfference: 0.058651
Finish 7 training epoch, Acc: 0.781967
Finish party 1 test risk difference 0.059173
Party 1 accuracy 0.782289
Finish party 2 test risk difference 0.066313
Party 2 accuracy 0.782289
Finish 7 testing epoch, Acc: 0.782289
Finish 7 risk difference 0.062764
**********
epoch 8
tensor(0.4376, grad_fn=<MeanBackward0>)
Party 1 accuracy 0.783200
Party 1 accuracy 0.780867
Party 1 risk difference 0.056213
Party 2 risk difference 0.063453
global training risk dfference: 0.059834
Finish 8 training epoch, Acc: 0.782033
Finish party 1 test risk difference 0.059956
Party 1 accuracy 0.782026
Finish party 2 test risk difference 0.066703
Party 2 accuracy 0.782289
Finish 8 testing epoch, Acc: 0.782157
Finish 8 risk difference 0.063350
**********
epoch 9
tensor(0.4373, grad_fn=<MeanBackward0>)
Party 1 accuracy 0

tensor(0.4333, grad_fn=<MeanBackward0>)
Party 1 accuracy 0.782667
Party 1 accuracy 0.781400
Party 1 risk difference 0.065713
Party 2 risk difference 0.072787
global training risk dfference: 0.069252
Finish 5 training epoch, Acc: 0.782033
Finish party 1 test risk difference 0.064579
Party 1 accuracy 0.780844
Finish party 2 test risk difference 0.073592
Party 2 accuracy 0.782026
Finish 5 testing epoch, Acc: 0.781435
Finish 5 risk difference 0.069108
**********
epoch 6
tensor(0.4331, grad_fn=<MeanBackward0>)
Party 1 accuracy 0.782400
Party 1 accuracy 0.781467
Party 1 risk difference 0.066001
Party 2 risk difference 0.073279
global training risk dfference: 0.069642
Finish 6 training epoch, Acc: 0.781933
Finish party 1 test risk difference 0.064571
Party 1 accuracy 0.781238
Finish party 2 test risk difference 0.073981
Party 2 accuracy 0.781763
Finish 6 testing epoch, Acc: 0.781500
Finish 6 risk difference 0.069297
**********
epoch 7
tensor(0.4329, grad_fn=<MeanBackward0>)
Party 1 accuracy 0

tensor(0.4295, grad_fn=<MeanBackward0>)
Party 1 accuracy 0.784400
Party 1 accuracy 0.783800
Party 1 risk difference 0.067994
Party 2 risk difference 0.074362
global training risk dfference: 0.071180
Finish 3 training epoch, Acc: 0.784100
Finish party 1 test risk difference 0.071202
Party 1 accuracy 0.782946
Finish party 2 test risk difference 0.075360
Party 2 accuracy 0.784128
Finish 3 testing epoch, Acc: 0.783537
Finish 3 risk difference 0.073310
**********
epoch 4
tensor(0.4293, grad_fn=<MeanBackward0>)
Party 1 accuracy 0.784867
Party 1 accuracy 0.784000
Party 1 risk difference 0.068775
Party 2 risk difference 0.074246
global training risk dfference: 0.071512
Finish 4 training epoch, Acc: 0.784433
Finish party 1 test risk difference 0.070998
Party 1 accuracy 0.783209
Finish party 2 test risk difference 0.075555
Party 2 accuracy 0.783997
Finish 4 testing epoch, Acc: 0.783603
Finish 4 risk difference 0.073304
**********
epoch 5
tensor(0.4292, grad_fn=<MeanBackward0>)
Party 1 accuracy 0

tensor(0.4263, grad_fn=<MeanBackward0>)
Party 1 accuracy 0.791267
Party 1 accuracy 0.790400
Party 1 risk difference 0.070319
Party 2 risk difference 0.075533
global training risk dfference: 0.072928
Finish 1 training epoch, Acc: 0.790833
Finish party 1 test risk difference 0.072195
Party 1 accuracy 0.788990
Finish party 2 test risk difference 0.081201
Party 2 accuracy 0.790435
Finish 1 testing epoch, Acc: 0.789712
Finish 1 risk difference 0.076719
**********
epoch 2
tensor(0.4261, grad_fn=<MeanBackward0>)
Party 1 accuracy 0.791533
Party 1 accuracy 0.790667
Party 1 risk difference 0.070804
Party 2 risk difference 0.075407
global training risk dfference: 0.073108
Finish 2 training epoch, Acc: 0.791100
Finish party 1 test risk difference 0.072375
Party 1 accuracy 0.789647
Finish party 2 test risk difference 0.083118
Party 2 accuracy 0.792012
Finish 2 testing epoch, Acc: 0.790829
Finish 2 risk difference 0.077770
**********
epoch 3
tensor(0.4260, grad_fn=<MeanBackward0>)
Party 1 accuracy 0

tensor(0.4235, grad_fn=<MeanBackward0>)
Party 1 accuracy 0.802733
Party 1 accuracy 0.800533
Party 1 risk difference 0.087280
Party 2 risk difference 0.087570
global training risk dfference: 0.087429
Finish 9 training epoch, Acc: 0.801633
Finish party 1 test risk difference 0.081573
Party 1 accuracy 0.800946
Finish party 2 test risk difference 0.094111
Party 2 accuracy 0.801209
Finish 9 testing epoch, Acc: 0.801077
Finish 9 risk difference 0.087862
**********
epoch 10
tensor(0.4233, grad_fn=<MeanBackward0>)
Party 1 accuracy 0.803600
Party 1 accuracy 0.801333
Party 1 risk difference 0.088851
Party 2 risk difference 0.089541
global training risk dfference: 0.089200
Finish 10 training epoch, Acc: 0.802467
Finish party 1 test risk difference 0.083913
Party 1 accuracy 0.801340
Finish party 2 test risk difference 0.094875
Party 2 accuracy 0.802128
Finish 10 testing epoch, Acc: 0.801734
Finish 10 risk difference 0.089412


In [7]:
# with fair
for i in range(Iteration):
    # compute with the option "with fair" => Zafar et al.
    theta, bias = compute_model_parameter_fairness(weight, Sen_torch, ratio_fair, options[1], k)

**********
epoch 1
tensor(3.1944, grad_fn=<AddBackward0>)
Party 1 accuracy 0.804400
Party 1 accuracy 0.802400
Party 1 risk difference 0.090216
Party 2 risk difference 0.092103
global training risk dfference: 0.091164
Finish 1 training epoch, Acc: 0.803400
Finish party 1 test risk difference 0.423856
Party 1 accuracy 0.709762
Finish party 2 test risk difference 0.424053
Party 2 accuracy 0.696229
Finish 1 testing epoch, Acc: 0.702996
Finish 1 risk difference 0.424036
**********
epoch 2
tensor(1.2806, grad_fn=<AddBackward0>)
Party 1 accuracy 0.700067
Party 1 accuracy 0.704133
Party 1 risk difference 0.420519
Party 2 risk difference 0.433159
global training risk dfference: 0.426833
Finish 2 training epoch, Acc: 0.702100
Finish party 1 test risk difference 0.399808
Party 1 accuracy 0.694915
Finish party 2 test risk difference 0.391884
Party 2 accuracy 0.681908
Finish 2 testing epoch, Acc: 0.688412
Finish 2 risk difference 0.395946
**********
epoch 3
tensor(1.1541, grad_fn=<AddBackward0>)
Pa

tensor(0.7136, grad_fn=<AddBackward0>)
Party 1 accuracy 0.754333
Party 1 accuracy 0.759467
Party 1 risk difference 0.173324
Party 2 risk difference 0.155717
global training risk dfference: 0.164525
Finish 9 training epoch, Acc: 0.756900
Finish party 1 test risk difference 0.165684
Party 1 accuracy 0.759164
Finish party 2 test risk difference 0.170189
Party 2 accuracy 0.754303
Finish 9 testing epoch, Acc: 0.756734
Finish 9 risk difference 0.167964
**********
epoch 10
tensor(0.7086, grad_fn=<AddBackward0>)
Party 1 accuracy 0.755600
Party 1 accuracy 0.762333
Party 1 risk difference 0.169589
Party 2 risk difference 0.148022
global training risk dfference: 0.158810
Finish 10 training epoch, Acc: 0.758967
Finish party 1 test risk difference 0.162006
Party 1 accuracy 0.759164
Finish party 2 test risk difference 0.168810
Party 2 accuracy 0.756931
Finish 10 testing epoch, Acc: 0.758048
Finish 10 risk difference 0.165427
**********
epoch 1
tensor(0.7044, grad_fn=<AddBackward0>)
Party 1 accuracy 

tensor(0.6775, grad_fn=<AddBackward0>)
Party 1 accuracy 0.771733
Party 1 accuracy 0.775867
Party 1 risk difference 0.167819
Party 2 risk difference 0.150751
global training risk dfference: 0.159295
Finish 7 training epoch, Acc: 0.773800
Finish party 1 test risk difference 0.159003
Party 1 accuracy 0.769281
Finish party 2 test risk difference 0.176524
Party 2 accuracy 0.777165
Finish 7 testing epoch, Acc: 0.773223
Finish 7 risk difference 0.167770
**********
epoch 8
tensor(0.6769, grad_fn=<AddBackward0>)
Party 1 accuracy 0.773000
Party 1 accuracy 0.776733
Party 1 risk difference 0.168730
Party 2 risk difference 0.151352
global training risk dfference: 0.160052
Finish 8 training epoch, Acc: 0.774867
Finish party 1 test risk difference 0.159240
Party 1 accuracy 0.769281
Finish party 2 test risk difference 0.179637
Party 2 accuracy 0.777427
Finish 8 testing epoch, Acc: 0.773354
Finish 8 risk difference 0.169435
**********
epoch 9
tensor(0.6763, grad_fn=<AddBackward0>)
Party 1 accuracy 0.77

tensor(0.6703, grad_fn=<AddBackward0>)
Party 1 accuracy 0.787267
Party 1 accuracy 0.787800
Party 1 risk difference 0.181505
Party 2 risk difference 0.163616
global training risk dfference: 0.172572
Finish 5 training epoch, Acc: 0.787533
Finish party 1 test risk difference 0.178223
Party 1 accuracy 0.782157
Finish party 2 test risk difference 0.191114
Party 2 accuracy 0.789384
Finish 5 testing epoch, Acc: 0.785771
Finish 5 risk difference 0.184682
**********
epoch 6
tensor(0.6700, grad_fn=<AddBackward0>)
Party 1 accuracy 0.787400
Party 1 accuracy 0.788733
Party 1 risk difference 0.181340
Party 2 risk difference 0.164476
global training risk dfference: 0.172919
Finish 6 training epoch, Acc: 0.788067
Finish party 1 test risk difference 0.178027
Party 1 accuracy 0.782026
Finish party 2 test risk difference 0.191548
Party 2 accuracy 0.790304
Finish 6 testing epoch, Acc: 0.786165
Finish 6 risk difference 0.184797
**********
epoch 7
tensor(0.6698, grad_fn=<AddBackward0>)
Party 1 accuracy 0.78

tensor(0.6671, grad_fn=<AddBackward0>)
Party 1 accuracy 0.801267
Party 1 accuracy 0.801067
Party 1 risk difference 0.197062
Party 2 risk difference 0.180869
global training risk dfference: 0.188978
Finish 3 training epoch, Acc: 0.801167
Finish party 1 test risk difference 0.184188
Party 1 accuracy 0.795428
Finish party 2 test risk difference 0.199180
Party 2 accuracy 0.798450
Finish 3 testing epoch, Acc: 0.796939
Finish 3 risk difference 0.191701
**********
epoch 4
tensor(0.6669, grad_fn=<AddBackward0>)
Party 1 accuracy 0.802733
Party 1 accuracy 0.801867
Party 1 risk difference 0.196716
Party 2 risk difference 0.181819
global training risk dfference: 0.189280
Finish 4 training epoch, Acc: 0.802300
Finish party 1 test risk difference 0.185999
Party 1 accuracy 0.796873
Finish party 2 test risk difference 0.199808
Party 2 accuracy 0.798975
Finish 4 testing epoch, Acc: 0.797924
Finish 4 risk difference 0.192924
**********
epoch 5
tensor(0.6668, grad_fn=<AddBackward0>)
Party 1 accuracy 0.80

...

tensor(0.6637, grad_fn=<AddBackward0>)
Party 1 accuracy 0.819067
Party 1 accuracy 0.822133
Party 1 risk difference 0.204900
Party 2 risk difference 0.202598
global training risk dfference: 0.203753
Finish 7 training epoch, Acc: 0.820600
Finish party 1 test risk difference 0.201575
Party 1 accuracy 0.817632
Finish party 2 test risk difference 0.214846
Party 2 accuracy 0.819603
Finish 7 testing epoch, Acc: 0.818618
Finish 7 risk difference 0.208230
**********
epoch 8
tensor(0.6637, grad_fn=<AddBackward0>)
Party 1 accuracy 0.819067
Party 1 accuracy 0.822067
Party 1 risk difference 0.204506
Party 2 risk difference 0.202410
global training risk dfference: 0.203461
Finish 8 training epoch, Acc: 0.820567
Finish party 1 test risk difference 0.201575
Party 1 accuracy 0.817632
Finish party 2 test risk difference 0.215056
Party 2 accuracy 0.819866
Finish 8 testing epoch, Acc: 0.818749
Finish 8 risk difference 0.208333
**********
epoch 9
tensor(0.6637, grad_fn=<AddBackward0>)
Party 1 accuracy 0.81

tensor(0.6633, grad_fn=<AddBackward0>)
Party 1 accuracy 0.820267
Party 1 accuracy 0.823867
Party 1 risk difference 0.203697
Party 2 risk difference 0.200969
global training risk dfference: 0.202337
Finish 5 training epoch, Acc: 0.822067
Finish party 1 test risk difference 0.201665
Party 1 accuracy 0.820917
Finish party 2 test risk difference 0.211596
Party 2 accuracy 0.822625
Finish 5 testing epoch, Acc: 0.821771
Finish 5 risk difference 0.206657
**********
epoch 6
tensor(0.6633, grad_fn=<AddBackward0>)
Party 1 accuracy 0.820600
Party 1 accuracy 0.824067
Party 1 risk difference 0.203507
Party 2 risk difference 0.200978
global training risk dfference: 0.202247
Finish 6 training epoch, Acc: 0.822333
Finish party 1 test risk difference 0.200678
Party 1 accuracy 0.821443
Finish party 2 test risk difference 0.211806
Party 2 accuracy 0.822888
Finish 6 testing epoch, Acc: 0.822165
Finish 6 risk difference 0.206267
**********
epoch 7
tensor(0.6633, grad_fn=<AddBackward0>)
Party 1 accuracy 0.82

tensor(0.6630, grad_fn=<AddBackward0>)
Party 1 accuracy 0.821933
Party 1 accuracy 0.826333
Party 1 risk difference 0.201556
Party 2 risk difference 0.200549
global training risk dfference: 0.201057
Finish 3 training epoch, Acc: 0.824133
Finish party 1 test risk difference 0.198957
Party 1 accuracy 0.824070
Finish party 2 test risk difference 0.206505
Party 2 accuracy 0.823939
Finish 3 testing epoch, Acc: 0.824005
Finish 3 risk difference 0.202765
**********
epoch 4
tensor(0.6630, grad_fn=<AddBackward0>)
Party 1 accuracy 0.822067
Party 1 accuracy 0.826467
Party 1 risk difference 0.201967
Party 2 risk difference 0.200961
global training risk dfference: 0.201469
Finish 4 training epoch, Acc: 0.824267
Finish party 1 test risk difference 0.198957
Party 1 accuracy 0.824070
Finish party 2 test risk difference 0.206311
Party 2 accuracy 0.824070
Finish 4 testing epoch, Acc: 0.824070
Finish 4 risk difference 0.202667
**********
epoch 5
tensor(0.6630, grad_fn=<AddBackward0>)
Party 1 accuracy 0.82

tensor(0.6628, grad_fn=<AddBackward0>)
Party 1 accuracy 0.823067
Party 1 accuracy 0.826867
Party 1 risk difference 0.199672
Party 2 risk difference 0.196195
global training risk dfference: 0.197940
Finish 1 training epoch, Acc: 0.824967
Finish party 1 test risk difference 0.196429
Party 1 accuracy 0.825253
Finish party 2 test risk difference 0.203810
Party 2 accuracy 0.824727
Finish 1 testing epoch, Acc: 0.824990
Finish 1 risk difference 0.200152
**********
epoch 2
tensor(0.6628, grad_fn=<AddBackward0>)
Party 1 accuracy 0.823400
Party 1 accuracy 0.826733
Party 1 risk difference 0.199680
Party 2 risk difference 0.195998
global training risk dfference: 0.197846
Finish 2 training epoch, Acc: 0.825067
Finish party 1 test risk difference 0.196242
Party 1 accuracy 0.825778
Finish party 2 test risk difference 0.203420
Party 2 accuracy 0.824990
Finish 2 testing epoch, Acc: 0.825384
Finish 2 risk difference 0.199865
**********
epoch 3
tensor(0.6628, grad_fn=<AddBackward0>)
Party 1 accuracy 0.82

tensor(0.6626, grad_fn=<AddBackward0>)
Party 1 accuracy 0.824533
Party 1 accuracy 0.827533
Party 1 risk difference 0.193200
Party 2 risk difference 0.191662
global training risk dfference: 0.192437
Finish 9 training epoch, Acc: 0.826033
Finish party 1 test risk difference 0.193697
Party 1 accuracy 0.826698
Finish party 2 test risk difference 0.201114
Party 2 accuracy 0.826304
Finish 9 testing epoch, Acc: 0.826501
Finish 9 risk difference 0.197437
**********
epoch 10
tensor(0.6626, grad_fn=<AddBackward0>)
Party 1 accuracy 0.824600
Party 1 accuracy 0.827667
Party 1 risk difference 0.193406
Party 2 risk difference 0.191769
global training risk dfference: 0.192593
Finish 10 training epoch, Acc: 0.826133
Finish party 1 test risk difference 0.194097
Party 1 accuracy 0.826830
Finish party 2 test risk difference 0.200530
Party 2 accuracy 0.826435
Finish 10 testing epoch, Acc: 0.826633
Finish 10 risk difference 0.197345


In [8]:
# agnostic fair
for i in range(Iteration):
    # compute with the option "with fair" => Zafar et al.
    theta, bias = compute_model_parameter_fairness(weight, Sen_torch, ratio_fair, options[2], k)

**********
epoch 1
tensor(0.6478, grad_fn=<AddBackward0>)
Party 1 accuracy 0.824733
Party 1 accuracy 0.827533
Party 1 risk difference 0.193513
Party 2 risk difference 0.191465
global training risk dfference: 0.192495
Finish 1 training epoch, Acc: 0.826133
Finish party 1 test risk difference 0.165486
Party 1 accuracy 0.829457
Finish party 2 test risk difference 0.173220
Party 2 accuracy 0.829457
Finish 1 testing epoch, Acc: 0.829457
Finish 1 risk difference 0.169386
**********
epoch 2
tensor(0.6437, grad_fn=<AddBackward0>)
Party 1 accuracy 0.829667
Party 1 accuracy 0.831467
Party 1 risk difference 0.165084
Party 2 risk difference 0.164379
global training risk dfference: 0.164738
Finish 2 training epoch, Acc: 0.830567
Finish party 1 test risk difference 0.148736
Party 1 accuracy 0.826961
Finish party 2 test risk difference 0.151534
Party 2 accuracy 0.826041
Finish 2 testing epoch, Acc: 0.826501
Finish 2 risk difference 0.150161
**********
epoch 3
tensor(0.6425, grad_fn=<AddBackward0>)
Pa

tensor(0.6404, grad_fn=<AddBackward0>)
Party 1 accuracy 0.831333
Party 1 accuracy 0.828067
Party 1 risk difference 0.153852
Party 2 risk difference 0.152582
global training risk dfference: 0.153220
Finish 9 training epoch, Acc: 0.829700
Finish party 1 test risk difference 0.156009
Party 1 accuracy 0.829195
Finish party 2 test risk difference 0.164251
Party 2 accuracy 0.830377
Finish 9 testing epoch, Acc: 0.829786
Finish 9 risk difference 0.160148
**********
epoch 10
tensor(0.6404, grad_fn=<AddBackward0>)
Party 1 accuracy 0.831667
Party 1 accuracy 0.828333
Party 1 risk difference 0.153688
Party 2 risk difference 0.155287
global training risk dfference: 0.154491
Finish 10 training epoch, Acc: 0.830000
Finish party 1 test risk difference 0.156775
Party 1 accuracy 0.829457
Finish party 2 test risk difference 0.164746
Party 2 accuracy 0.830246
Finish 10 testing epoch, Acc: 0.829852
Finish 10 risk difference 0.160785
**********
epoch 1
tensor(0.6403, grad_fn=<AddBackward0>)
Party 1 accuracy 

tensor(0.6398, grad_fn=<AddBackward0>)
Party 1 accuracy 0.831200
Party 1 accuracy 0.832400
Party 1 risk difference 0.169700
Party 2 risk difference 0.171579
global training risk dfference: 0.170642
Finish 7 training epoch, Acc: 0.831800
Finish party 1 test risk difference 0.171837
Party 1 accuracy 0.833268
Finish party 2 test risk difference 0.171009
Party 2 accuracy 0.832873
Finish 7 testing epoch, Acc: 0.833071
Finish 7 risk difference 0.171456
**********
epoch 8
tensor(0.6398, grad_fn=<AddBackward0>)
Party 1 accuracy 0.831333
Party 1 accuracy 0.832333
Party 1 risk difference 0.169363
Party 2 risk difference 0.171856
global training risk dfference: 0.170612
Finish 8 training epoch, Acc: 0.831833
Finish party 1 test risk difference 0.172416
Party 1 accuracy 0.833005
Finish party 2 test risk difference 0.170979
Party 2 accuracy 0.832611
Finish 8 testing epoch, Acc: 0.832808
Finish 8 risk difference 0.171732
**********
epoch 9
tensor(0.6398, grad_fn=<AddBackward0>)
Party 1 accuracy 0.83

tensor(0.6396, grad_fn=<AddBackward0>)
Party 1 accuracy 0.830867
Party 1 accuracy 0.833000
Party 1 risk difference 0.175008
Party 2 risk difference 0.177140
global training risk dfference: 0.176077
Finish 5 training epoch, Acc: 0.831933
Finish party 1 test risk difference 0.175987
Party 1 accuracy 0.831034
Finish party 2 test risk difference 0.175847
Party 2 accuracy 0.834844
Finish 5 testing epoch, Acc: 0.832939
Finish 5 risk difference 0.175945
**********
epoch 6
tensor(0.6395, grad_fn=<AddBackward0>)
Party 1 accuracy 0.830800
Party 1 accuracy 0.832800
Party 1 risk difference 0.174893
Party 2 risk difference 0.176719
global training risk dfference: 0.175809
Finish 6 training epoch, Acc: 0.831800
Finish party 1 test risk difference 0.176183
Party 1 accuracy 0.831165
Finish party 2 test risk difference 0.175443
Party 2 accuracy 0.834976
Finish 6 testing epoch, Acc: 0.833071
Finish 6 risk difference 0.175842
**********
epoch 7
tensor(0.6395, grad_fn=<AddBackward0>)
Party 1 accuracy 0.83

tensor(0.6394, grad_fn=<AddBackward0>)
Party 1 accuracy 0.830000
Party 1 accuracy 0.832533
Party 1 risk difference 0.176375
Party 2 risk difference 0.177256
global training risk dfference: 0.176819
Finish 3 training epoch, Acc: 0.831267
Finish party 1 test risk difference 0.177495
Party 1 accuracy 0.830903
Finish party 2 test risk difference 0.178678
Party 2 accuracy 0.834976
Finish 3 testing epoch, Acc: 0.832939
Finish 3 risk difference 0.178115
**********
epoch 4
tensor(0.6394, grad_fn=<AddBackward0>)
Party 1 accuracy 0.830000
Party 1 accuracy 0.832333
Party 1 risk difference 0.176572
Party 2 risk difference 0.177049
global training risk dfference: 0.176815
Finish 4 training epoch, Acc: 0.831167
Finish party 1 test risk difference 0.177691
Party 1 accuracy 0.830771
Finish party 2 test risk difference 0.178678
Party 2 accuracy 0.834976
Finish 4 testing epoch, Acc: 0.832873
Finish 4 risk difference 0.178213
**********
epoch 5
tensor(0.6394, grad_fn=<AddBackward0>)
Party 1 accuracy 0.82

...

tensor(0.6390, grad_fn=<AddBackward0>)
Party 1 accuracy 0.828667
Party 1 accuracy 0.831933
Party 1 risk difference 0.178022
Party 2 risk difference 0.173697
global training risk dfference: 0.175865
Finish 7 training epoch, Acc: 0.830300
Finish party 1 test risk difference 0.176018
Party 1 accuracy 0.829852
Finish party 2 test risk difference 0.175520
Party 2 accuracy 0.835633
Finish 7 testing epoch, Acc: 0.832742
Finish 7 risk difference 0.175801
**********
epoch 8
tensor(0.6390, grad_fn=<AddBackward0>)
Party 1 accuracy 0.828800
Party 1 accuracy 0.831933
Party 1 risk difference 0.177825
Party 2 risk difference 0.173697
global training risk dfference: 0.175767
Finish 8 training epoch, Acc: 0.830367
Finish party 1 test risk difference 0.176214
Party 1 accuracy 0.829983
Finish party 2 test risk difference 0.176104
Party 2 accuracy 0.835238
Finish 8 testing epoch, Acc: 0.832611
Finish 8 risk difference 0.176191
**********
epoch 9
tensor(0.6390, grad_fn=<AddBackward0>)
Party 1 accuracy 0.82

tensor(0.6390, grad_fn=<AddBackward0>)
Party 1 accuracy 0.828533
Party 1 accuracy 0.831800
Party 1 risk difference 0.177488
Party 2 risk difference 0.173661
global training risk dfference: 0.175580
Finish 5 training epoch, Acc: 0.830167
Finish party 1 test risk difference 0.176002
Party 1 accuracy 0.830114
Finish party 2 test risk difference 0.173289
Party 2 accuracy 0.835501
Finish 5 testing epoch, Acc: 0.832808
Finish 5 risk difference 0.174681
**********
epoch 6
tensor(0.6390, grad_fn=<AddBackward0>)
Party 1 accuracy 0.828533
Party 1 accuracy 0.831800
Party 1 risk difference 0.177488
Party 2 risk difference 0.173661
global training risk dfference: 0.175580
Finish 6 training epoch, Acc: 0.830167
Finish party 1 test risk difference 0.175806
Party 1 accuracy 0.829983
Finish party 2 test risk difference 0.172690
Party 2 accuracy 0.835764
Finish 6 testing epoch, Acc: 0.832873
Finish 6 risk difference 0.174285
**********
epoch 7
tensor(0.6390, grad_fn=<AddBackward0>)
Party 1 accuracy 0.82

tensor(0.6389, grad_fn=<AddBackward0>)
Party 1 accuracy 0.828267
Party 1 accuracy 0.832067
Party 1 risk difference 0.177973
Party 2 risk difference 0.174234
global training risk dfference: 0.176110
Finish 3 training epoch, Acc: 0.830167
Finish party 1 test risk difference 0.176393
Party 1 accuracy 0.830114
Finish party 2 test risk difference 0.172870
Party 2 accuracy 0.836290
Finish 3 testing epoch, Acc: 0.833202
Finish 3 risk difference 0.174669
**********
epoch 4
tensor(0.6389, grad_fn=<AddBackward0>)
Party 1 accuracy 0.828200
Party 1 accuracy 0.832067
Party 1 risk difference 0.178072
Party 2 risk difference 0.174234
global training risk dfference: 0.176159
Finish 4 training epoch, Acc: 0.830133
Finish party 1 test risk difference 0.176393
Party 1 accuracy 0.830114
Finish party 2 test risk difference 0.172675
Party 2 accuracy 0.836158
Finish 4 testing epoch, Acc: 0.833136
Finish 4 risk difference 0.174572
**********
epoch 5
tensor(0.6389, grad_fn=<AddBackward0>)
Party 1 accuracy 0.82

tensor(0.6389, grad_fn=<AddBackward0>)
Party 1 accuracy 0.828533
Party 1 accuracy 0.831867
Party 1 risk difference 0.178549
Party 2 risk difference 0.174118
global training risk dfference: 0.176339
Finish 1 training epoch, Acc: 0.830200
Finish party 1 test risk difference 0.176385
Party 1 accuracy 0.830246
Finish party 2 test risk difference 0.172062
Party 2 accuracy 0.835764
Finish 1 testing epoch, Acc: 0.833005
Finish 1 risk difference 0.174262
**********
epoch 2
tensor(0.6389, grad_fn=<AddBackward0>)
Party 1 accuracy 0.828467
Party 1 accuracy 0.831733
Party 1 risk difference 0.178344
Party 2 risk difference 0.174118
global training risk dfference: 0.176237
Finish 2 training epoch, Acc: 0.830100
Finish party 1 test risk difference 0.176385
Party 1 accuracy 0.830246
Finish party 2 test risk difference 0.172062
Party 2 accuracy 0.835764
Finish 2 testing epoch, Acc: 0.833005
Finish 2 risk difference 0.174262
**********
epoch 3
tensor(0.6389, grad_fn=<AddBackward0>)
Party 1 accuracy 0.82

tensor(0.6388, grad_fn=<AddBackward0>)
Party 1 accuracy 0.828467
Party 1 accuracy 0.831733
Party 1 risk difference 0.178829
Party 2 risk difference 0.174512
global training risk dfference: 0.176676
Finish 9 training epoch, Acc: 0.830100
Finish party 1 test risk difference 0.176777
Party 1 accuracy 0.830246
Finish party 2 test risk difference 0.172256
Party 2 accuracy 0.835633
Finish 9 testing epoch, Acc: 0.832939
Finish 9 risk difference 0.174554
**********
epoch 10
tensor(0.6388, grad_fn=<AddBackward0>)
Party 1 accuracy 0.828467
Party 1 accuracy 0.831667
Party 1 risk difference 0.178829
Party 2 risk difference 0.174413
global training risk dfference: 0.176627
Finish 10 training epoch, Acc: 0.830067
Finish party 1 test risk difference 0.176777
Party 1 accuracy 0.830246
Finish party 2 test risk difference 0.172256
Party 2 accuracy 0.835633
Finish 10 testing epoch, Acc: 0.832939
Finish 10 risk difference 0.174554
